In [0]:
# !pip -q uninstall -y tensorflow
# !pip -q install tf-nightly

# Download 
!wget https://thehill.com/sites/default/files/styles/thumb_small_article/public/blogs/obama2_0.jpg
!wget https://i.cbc.ca/1.5044573.1551844227!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/obama.jpg

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
%matplotlib inline

import tensorflow as tf
from datetime import datetime

LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]

label_index = LABEL_NAMES.index('person')

# Model conversion
## Convert from Deeplabv3's pre-trained model

In [0]:
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
!tar -xvzf deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz

In [0]:
frozen_model_file = 'deeplabv3_mnv2_pascal_train_aug/frozen_inference_graph.pb'
tflite_file = 'deeplabv3_mnv2_pascal_train_aug/deeplabv3_mnv2_pascal_train_aug.tflite'

converter = tf.lite.TFLiteConverter.from_frozen_graph(graph_def_file=frozen_model_file,
                                  input_arrays=["sub_7"],
                                  output_arrays=["ArgMax"],
                                  # input_shapes={"sub_7": [1, 513, 513, 3]}
                                  )
# converter.inference_input_type = tf.int8
# converter.default_ranges_stats

tflite_model = converter.convert()
f = open(tflite_file, "wb")
f.write(tflite_model)
f.close()


In [0]:
# Download the TF Lite model
from google.colab import files
files.download(tflite_file)

## Inference using converted TF Lite Deeplabv3 model

In [0]:
image_file = 'obama2_0.jpg'

input_image_size = 513

input_image = np.array(Image.open(image_file).resize([input_image_size, input_image_size]), dtype=np.float32)
# input_image = np.expand_dims((input_image - 127.5) / 127.5, 0)
input_image = np.expand_dims(input_image / 255.0, 0)
# input_image = np.expand_dims(input_image, 0)

interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

print(interpreter.get_input_details()[0])
interpreter.set_tensor(interpreter.get_input_details()[0]["index"], input_image)
start_time = datetime.now()
interpreter.invoke()
finish_time = datetime.now()
print('Latency = %d ms' % ((finish_time - start_time).total_seconds() * 1000))
result = interpreter.tensor(interpreter.get_output_details()[0]["index"])()

print(np.max(result[0]))

plt.figure(figsize=(20,10))
# plt.imshow(input_image_original)
# plt.imshow(resized_result, alpha=0.5)
# plt.imshow(input_image_original.resize([input_image_size, input_image_size]))
# plt.imshow(result, alpha=0.5)
# plt.imshow(resized_result, interpolation='nearest')
plt.imshow(input_image[0])
plt.imshow(result[0] / np.max(result[0]), alpha=0.5)

plt.show()


# TF Lite's published image segmentation model

In [0]:
!wget https://storage.googleapis.com/download.tensorflow.org/models/tflite/gpu/deeplabv3_257_mv_gpu.tflite

In [0]:
from google.colab import files
files.upload()

In [0]:
model_file = 'deeplabv3_257_mv_gpu.tflite'
# model_file = 'deeplabv3.tflite'
# model_file = 'deeplabv3_small.tflite'

# image_file = 'obama2_0.jpg'
image_file = 'obama.jpg'

# Load TF Lite model
interpreter = tf.lite.Interpreter(model_path=model_file)
input_image_size = interpreter.get_input_details()[0]["shape"][1]

# Pre-process input image
input_image_original = Image.open(image_file)
input_image = np.array(input_image_original.resize([input_image_size, input_image_size]), dtype=np.float32)
input_image = np.expand_dims((input_image / 128.0) - 1.0, 0)
# input_image = np.expand_dims(input_image / 255.0, 0)
# input_image = np.expand_dims(input_image, 0)

# Run inference
interpreter.allocate_tensors()
interpreter.set_tensor(interpreter.get_input_details()[0]["index"], input_image)
start_time = datetime.now()
interpreter.invoke()
finish_time = datetime.now()
print('Latency = %d ms' % ((finish_time - start_time).total_seconds() * 1000))
logits = interpreter.tensor(interpreter.get_output_details()[0]["index"])()

# Raw segmentation output from model, same size with TF Lite model output
result = np.argmax(logits[0], axis=2).astype(np.int8)

# Resize segmentation output to original image
im = Image.fromarray(result)
resized_result = im.resize(input_image_original.size, resample=Image.NEAREST)
print(np.shape(resized_result))

# Show result
f = plt.figure(figsize=(20,10))
f.add_subplot(1, 2, 1)
plt.imshow(input_image_original)
plt.imshow(resized_result, alpha=0.5)
f.add_subplot(1, 2, 2)
plt.imshow(input_image_original.resize([input_image_size, input_image_size]))
plt.imshow(result, alpha=0.5)
# plt.imshow(resized_result, interpolation='nearest')
plt.show()

